<a href="https://colab.research.google.com/github/fevdaele/Machine_Learning_course_UGent_D012554_kaggle/blob/master/PredictionsFemke_12_05_2020_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt;
import numpy as np;
import pandas as pd;
import seaborn as sns;
sns.set_context("notebook", font_scale=1.4);
sns.set_style("whitegrid");
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **Load train and dataset**

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

### **Data pre processing**

In [0]:
data_train = trainset.copy()
label_column = data_train.pop('label')

data_test = testset.copy()
index_column = data_test.pop('index')

**Scaling of trainset**

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train = scaler.fit_transform(data_train)


#plt.figure(figsize=(12,8))
#trainset_scaled.boxplot(vert=False)
#plt.show()

**Scaling of testset**

In [0]:
data_test = scaler.transform(data_test)
#plt.figure(figsize=(12,8))
#testset_scaled.boxplot(vert=False)
#plt.show()


### ***Model selection and validation***

In [9]:
from sklearn.svm import SVC

y = label_column
X = data_train

model = SVC(kernel='rbf', gamma=1 , C=1)

score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)


0.9581062148860315


### ***Model optimization***

In [10]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import metrics

X = data_train
y = label_column


search_space_C = [0.001, 0.01, 0.1, 1, 10]
search_space_gamma = [1, 2, 3, 4, 5, 6]
param_grid = {'gamma': search_space_gamma, 'C' : search_space_C}
grid_search = GridSearchCV(model, param_grid, scoring = 'roc_auc', cv=10)
grid_search.fit(X, y)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
{'C': 10, 'gamma': 1}
0.9625710354884667


**Define and validate model with optimized hyper parameter**

In [11]:
model = SVC(kernel='rbf', gamma=1, C=10, probability=True)
score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)

0.9625710354884667


### ***Fit model on trainset***

In [12]:
model.fit(X,y)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### ***Make predictions***

In [13]:
predictions = model.predict_proba(data_test)
print(predictions)

[[0.9962682  0.0037318 ]
 [0.62858231 0.37141769]
 [0.99262406 0.00737594]
 ...
 [0.11175654 0.88824346]
 [0.05209282 0.94790718]
 [0.99481326 0.00518674]]


**Create new dataframe to load predictions**

In [14]:
PredictionsFemke = pd.DataFrame(columns=['index','label'])
PredictionsFemke['index'] = index_column
PredictionsFemke['label'] = predictions[:,1]

print(PredictionsFemke)

       index     label
0          0  0.003732
1          1  0.371418
2          2  0.007376
3          3  0.996818
4          4  0.946966
...      ...       ...
12887  12887  0.118394
12888  12888  0.975516
12889  12889  0.888243
12890  12890  0.947907
12891  12891  0.005187

[12892 rows x 2 columns]


### **Save predictions in csv file**

In [0]:
filename = "PredictionsFemke12052020_version2.csv"

#make sure to not write the Pandas index column (index=False)
PredictionsFemke.to_csv(filename,index=False)